<a href="https://colab.research.google.com/github/PushpalathaSK/AI_Practice/blob/main/imputer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset suggestion: Titanic, House Prices, or any dataset with NaN values.


In [ ]:
from seaborn import load_dataset

data=load_dataset('titanic')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


Load the dataset and identify columns with missing values.


In [ ]:
data.isnull().sum()

,0
survived,0
pclass,0
sex,0
age,177
sibsp,0
parch,0
fare,0
embarked,2
class,0
who,0


In [ ]:
from sklearn.model_selection import train_test_split
X=data.drop('survived', axis=1)
y=data['survived']

Mean / Median / Mode imputation


In [ ]:
from sklearn.impute import SimpleImputer
data_simple=X.copy()
for col in data_simple.columns:
  if data_simple[col].isnull().sum()>0:
    if data_simple[col].dtype in ['object','category','bool']:
      imputer=SimpleImputer(strategy='most_frequent')
    else:
      imputer=SimpleImputer(strategy='mean')
    data_simple[[col]] = imputer.fit_transform(data_simple[[col]])



for col in data_simple.columns:
    if data_simple[col].dtype in ['object', 'category', 'bool']:
        le = LabelEncoder()
        data_simple[col] = le.fit_transform(data_simple[col].astype(str))


KNN imputation (KNNImputer)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
data_knn=X.copy()

for col in data_knn.columns:
  if data_knn[col].dtype in ['object','category','bool']:
    le = LabelEncoder()
    data_knn[col]=le.fit_transform(data_knn[col].astype(str))

imputer_knn = KNNImputer(n_neighbors=5)
data_knn=pd.DataFrame(imputer_knn.fit_transform(data_knn),columns=X.columns)

Iterative (predictive) imputation (IterativeImputer)


In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

data_imp=X.copy()
for col in data_imp.columns:
  if data_imp[col].dtype in ['object','category','bool']:
    le = LabelEncoder()
    data_imp[col]=le.fit_transform(data_imp[col].astype(str))
imputer=IterativeImputer(random_state=42)
data_imp=pd.DataFrame(imputer.fit_transform(data_imp),columns=X.columns)

Compare how each technique affects model accuracy (use Logistic Regression or Random Forest).


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from  sklearn.metrics import classification_report,accuracy_score

data_X={
    'Simple Imputer':data_simple,
    'KNN Imputer':data_knn,
    'Iterative Imputer':data_imp
}
model=RandomForestClassifier(random_state=42,max_depth=2)
for name,data in data_X.items():
  X_train, X_test, y_train, y_test=train_test_split(data, y, test_size=0.2, random_state=42)
  model.fit(X_train, y_train)
  y_pred=model.predict(X_test)
  print(name,"\n")
  print(accuracy_score(y_test,y_pred))
  print(classification_report(y_test, y_pred))


Simple Imputer 

0.9720670391061452
              precision    recall  f1-score   support

           0       0.95      1.00      0.98       105
           1       1.00      0.93      0.97        74

    accuracy                           0.97       179
   macro avg       0.98      0.97      0.97       179
weighted avg       0.97      0.97      0.97       179

KNN Imputer 

0.9664804469273743
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       105
           1       0.99      0.93      0.96        74

    accuracy                           0.97       179
   macro avg       0.97      0.96      0.97       179
weighted avg       0.97      0.97      0.97       179

Iterative Imputer 

0.9664804469273743
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       105
           1       0.99      0.93      0.96        74

    accuracy                           0.97       179
   macro avg       0

In [ ]:
# ==============================
# Handling Missing Values (Titanic)
# ==============================

from seaborn import load_dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

# ==============================
# 1️⃣ Load Dataset
# ==============================
data = load_dataset('titanic')
print(data.info())

# Separate features and target
X = data.drop('survived', axis=1)
y = data['survived']

# ==============================
# 2️⃣ Simple Imputer
# ==============================
data_simple = X.copy()
num_cols = data_simple.select_dtypes(exclude=['object', 'category', 'bool']).columns
cat_cols = data_simple.select_dtypes(include=['object', 'category', 'bool']).columns

num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

data_simple[num_cols] = num_imputer.fit_transform(data_simple[num_cols])
data_simple[cat_cols] = cat_imputer.fit_transform(data_simple[cat_cols])

# ==============================
# 3️⃣ KNN Imputer
# ==============================
data_knn = X.copy()
le = LabelEncoder()

# Encode categorical variables before KNN imputation
for col in data_knn.columns:
    if data_knn[col].dtype in ['object', 'category', 'bool']:
        data_knn[col] = le.fit_transform(data_knn[col].astype(str))

imputer_knn = KNNImputer(n_neighbors=5)
data_knn = pd.DataFrame(imputer_knn.fit_transform(data_knn), columns=X.columns)

# ==============================
# 4️⃣ Iterative Imputer
# ==============================
data_imp = X.copy()

# Encode categorical variables before Iterative imputation
for col in data_imp.columns:
    if data_imp[col].dtype in ['object', 'category', 'bool']:
        data_imp[col] = le.fit_transform(data_imp[col].astype(str))

imputer_iter = IterativeImputer(random_state=42)
data_imp = pd.DataFrame(imputer_iter.fit_transform(data_imp), columns=X.columns)

# ==============================
# 5️⃣ Model Training and Evaluation
# ==============================
data_versions = {
    'Simple Imputer': data_simple,
    'KNN Imputer': data_knn,
    'Iterative Imputer': data_imp
}

model = RandomForestClassifier(random_state=42)
accuracies = {}

for name, data_version in data_versions.items():
    X_train, X_test, y_train, y_test = train_test_split(
        data_version, y, test_size=0.2, random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies[name] = acc

    print("=" * 50)
    print(f"{name} Results:")
    print(f"Accuracy: {acc:.4f}\n")
    print(classification_report(y_test, y_pred))

# ==============================
# 6️⃣ Visual Comparison
# ==============================
plt.figure(figsize=(7, 5))
plt.bar(accuracies.keys(), accuracies.values())
plt.title("Model Accuracy after Different Imputation Methods")
plt.xlabel("Imputation Technique")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.show()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
None


ValueError: could not convert string to float: 'male'